**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [9]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-2mzpllup
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-2mzpllup
  Resolved https://github.com/Kaggle/learntools.git to commit ab3f19a77b8a92c7337c9fd15d84602e7320a077
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268981 sha256=15569cbf906e391af09c4ba5c7e53ae973707bcbcc56311e6f5d17740b762bc2
  Stored in directory: /tmp/pip-ephem-wheel-cache-3w2ae1ko/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [10]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,4dec1e9d50377bd56b38f32ba16257fe58eb33c7,daa87fd285aad3d29e8999316561c9c5dcb69c1be4bac2...,2019-04-07 00:15:00+00:00,2019-04-07 00:30:00+00:00,660,0.2,NaN,NaN,NaN,NaN,...,0.0,15.90,Credit Card,Taxi Affiliation Services,NaN,NaN,None,NaN,NaN,None
1,8528eb12fc48368ffc8217fe3296ef0f21651751,3129d0698b6fb9b549bb86967032b957abbfb485f80fc7...,2019-04-07 00:15:00+00:00,2019-04-07 00:30:00+00:00,1020,3.2,NaN,NaN,NaN,NaN,...,1.0,13.75,Cash,Taxi Affiliation Services,NaN,NaN,None,NaN,NaN,None
2,3733affa945a548529a3c24f4b2007f2cf544999,de0c7465b6b6befec9c1ed6f7f3290706f5fae5c7981e3...,2019-04-07 00:15:00+00:00,2019-04-07 00:30:00+00:00,420,0.0,NaN,1.703108e+10,NaN,8.0,...,0.0,6.25,Credit Card,Blue Ribbon Taxi Association Inc.,NaN,NaN,None,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365)
3,74cf5f4f466160b988a38819c8b093e6726a4dee,88f8209072b1c11e826a6b860b5f6c8d202fa2c1e28101...,2019-04-07 00:15:00+00:00,2019-04-07 00:30:00+00:00,300,0.4,NaN,1.703132e+10,NaN,32.0,...,1.0,6.00,Cash,KOAM Taxi Association,NaN,NaN,None,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918)
4,7cc91562ccba52e19a5eefc1513cea482d16706a,f752844e8b31c74a28770eca091af67c2611e7c1cf6be7...,2019-04-07 00:15:00+00:00,2019-04-07 00:30:00+00:00,420,3.3,NaN,NaN,NaN,NaN,...,0.0,10.75,Cash,Taxi Affiliation Services,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [11]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-02-16,90901.142857
1,2016-01-02,72801.000000
2,2016-02-13,92821.571429
3,2016-03-30,74274.400000
4,2016-01-11,83323.714286


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [12]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [13]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,61.0,2013-10-03 06:00:00+00:00,2013-10-03 06:15:00+00:00,1
1,61.0,2013-10-03 09:00:00+00:00,2013-10-03 09:30:00+00:00,2
2,61.0,2013-10-03 14:30:00+00:00,2013-10-03 14:30:00+00:00,3
3,52.0,2013-10-03 20:30:00+00:00,2013-10-03 21:00:00+00:00,1
4,76.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [14]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/zzHgmlx.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [15]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,0cebd4102de9b52a9264f60e46d4adcf7369a6e9ec9e0c...,2013-10-03 13:15:00+00:00,2013-10-03 13:15:00+00:00,165.0
1,1300fb7d123f7fb4a30c2b8fa76f7bce21128fb77728b5...,2013-10-03 20:30:00+00:00,2013-10-03 21:00:00+00:00,390.0
2,199cdc36e88754114b867e974f41cfb3252a6755a55d27...,2013-10-03 18:00:00+00:00,2013-10-03 18:00:00+00:00,210.0
3,1ab67f67f06ece209e20c9001a487153cf1f450f8b6ebf...,2013-10-03 06:15:00+00:00,2013-10-03 06:30:00+00:00,270.0
4,26ae60b930554c459e1f27da35b2958bc47ef65dad856e...,2013-10-03 18:00:00+00:00,2013-10-03 18:00:00+00:00,255.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [17]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*